# **Customer Lifetime Value**

CLTV = (Customer Value / Churn Rate) * Profit Margin

Customer Value = Purchase Frequency * Average Order Value

CLTV = Expected Number of Transaction * Expected Average Profit

CLTV = BG / NBD Model * Gamma Gamma Submodel 

# Data
This Online Retail II data set contains all the transactions occurring for a UK-based and registered, non-store online retail between 01/12/2009 and 09/12/2011.


**Variables**
1. **Invoice:** Invoice number, unique identifier variable for each transaction. Refund invoice numbers starts with "C"
2. **StockCode:** Unique product code
3. **Description:** Product name
4. **Quantity:** The number of product in the invoice
5. **InvoiceDate:** Date and time of the purchase
6. **Price:** Unit price of a product (in terms of Sterlin)
7. **CustomerID:** Unique customer identifier
8. **Country:** Residential country of customers

In [2]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [3]:
# Reading dataset
df_ = pd.read_excel('online_retail_II.xlsx', sheet_name = 'Year 2010-2011')

In [4]:
# Copying the dataset
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom


In [5]:
# Checking numerical variables
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541910.0000,9.5522,218.0810,-80995.0000,1.0000,3.0000,10.0000,80995.0000
Price,541910.0000,4.6111,96.7598,-11062.0600,1.2500,2.0800,4.1300,38970.0000
Customer ID,406830.0000,15287.6842,1713.6031,12346.0000,13953.0000,15152.0000,16791.0000,18287.0000


In [6]:
# Checking null variables 
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [7]:
# Removing null observations
df.dropna(inplace=True)

In [8]:
# Removing canceled orders
df = df[~df['Invoice'].astype(str).str.contains('C', na=False)]

In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397925.0000,13.0218,180.4200,1.0000,2.0000,6.0000,12.0000,80995.0000
Price,397925.0000,3.1162,22.0968,0.0000,1.2500,1.9500,3.7500,8142.7500
Customer ID,397925.0000,15294.3086,1713.1727,12346.0000,13969.0000,15159.0000,16795.0000,18287.0000


In [11]:
df = df[df['Quantity'] > 0]

In [12]:
df = df[df['Price'] > 0]

In [13]:
# to suppress outliers
def outlier_thresholds(dataframe, variable):
  quartile1 = dataframe[variable].quantile(0.01)
  quartile3 = dataframe[variable].quantile(0.99)
  interquantile_range = quartile3 - quartile1
  up_limit = quartile3 + 1.5 * interquantile_range
  low_limit = quartile1 - 1.5 * interquantile_range
  return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
  low_limit, up_limit = outlier_thresholds(dataframe, variable)#
#  dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
  dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [14]:
replace_with_thresholds(df, 'Quantity')

In [15]:
replace_with_thresholds(df, 'Price')

In [16]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397885.0000,11.8308,25.5231,1.0000,2.0000,6.0000,12.0000,298.5000
Price,397885.0000,2.8935,3.2272,0.0010,1.2500,1.9500,3.7500,37.0600
Customer ID,397885.0000,15294.4169,1713.1444,12346.0000,13969.0000,15159.0000,16795.0000,18287.0000


In [17]:
# Calculating total price per transaction 
df['TotalPrice'] = df['Quantity'] * df['Price']

In [18]:
today_date = dt.datetime(2011,12,11)

# R, T, F, M

In [19]:
# Recency: The age of the customer at the time of their last purchase.
# T : The age of the customer since the date of a customer's first purchase to the current date.
# Monetary: The average total sales of the customer.
# Frequency: Number of purchases/transactions.

cltv_df = df.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max() - date.min()).days,
                                                         lambda date: (today_date - date.min()).days],
                                         'Invoice': lambda num: num.nunique(),
                                         'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

In [20]:
cltv_df.head()# işlem başına ortalama kazanç
cltv_df['monetary'] = cltv_df['monetary'] / cltv_df['frequency']

InvoiceDate             Invoice TotalPrice
             <lambda_0> <lambda_1> <lambda>   <lambda>
Customer ID                                           
12346.0000            0        326        1   310.4400
12347.0000          365        368        7  4310.0000
12348.0000          282        359        4  1770.7800
12349.0000            0         19        1  1491.7200
12350.0000            0        311        1   331.4600

In [21]:
cltv_df.columns = cltv_df.columns.droplevel(0)

In [22]:
cltv_df.columns = ['recency','T','frequency','monetary']

In [23]:
cltv_df.head()

,recency,T,frequency,monetary
Customer ID,,,,
12346.0000,0,326,1,310.4400
12347.0000,365,368,7,4310.0000
12348.0000,282,359,4,1770.7800
12349.0000,0,19,1,1491.7200
12350.0000,0,311,1,331.4600


In [24]:
# Average earnings per trade
cltv_df['monetary'] = cltv_df['monetary'] / cltv_df['frequency']

In [25]:
cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

In [26]:
cltv_df['recency'] = cltv_df['recency'] / 7  # weekly

In [27]:
cltv_df['T'] = cltv_df['T'] / 7 # weekly

# BG-NBD Model

In [28]:
pip install lifetimes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 KB 11.1 MB/s eta 0:00:00


In [29]:
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

In [30]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(cltv_df['frequency'],
        cltv_df['recency'],
        cltv_df['T'])

<lifetimes.BetaGeoFitter: fitted with 2845 subjects, a: 0.12, alpha: 11.41, b: 2.49, r: 2.18>

In [31]:
# Estimation of expected number of transactions per customer during a month (4 weeks)
cltv_df['expected_purc_1_month'] = bgf.predict(4,
                                              cltv_df['frequency'],
                                              cltv_df['recency'],
                                              cltv_df['T'])

In [32]:
cltv_df.sort_values("expected_purc_1_month",ascending=False).head()

,recency,T,frequency,monetary,expected_purc_1_month
Customer ID,,,,,
12748.0000,53.1429,53.4286,209,154.9302,12.9633
14911.0000,53.1429,53.4286,201,691.7101,12.4722
17841.0000,53.0000,53.4286,124,330.1344,7.7398
13089.0000,52.2857,52.8571,97,606.3625,6.1330
14606.0000,53.1429,53.4286,93,130.1390,5.8399


# Gamma Gamma Model

In [34]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

ggf.fit(cltv_df['frequency'], cltv_df['monetary'])

<lifetimes.GammaGammaFitter: fitted with 2845 subjects, p: 3.79, q: 0.34, v: 3.73>

In [35]:
# Prediction of expected amount of average profit
cltv_df['expected_average_profit'] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                        cltv_df['monetary'])

In [36]:
cltv_df.head()

,recency,T,frequency,monetary,expected_purc_1_month,expected_average_profit
Customer ID,,,,,,
12347.0000,52.1429,52.5714,7,615.7143,0.5635,631.9123
12348.0000,40.2857,51.2857,4,442.6950,0.3668,463.7460
12352.0000,37.1429,42.4286,8,219.5425,0.7271,224.8868
12356.0000,43.1429,46.5714,3,937.1433,0.3435,995.9989
12358.0000,21.2857,21.5714,2,575.2100,0.4862,631.9022


# CLTV

In [38]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=3,  # 3 month
                                   freq='W', # frequency of T (W:weekly).
                                   discount_rate=0.01)

In [39]:
cltv.head()

Customer ID
12347.0000   1128.4477
12348.0000    538.8089
12352.0000    517.5000
12356.0000   1083.0903
12358.0000    966.6727
Name: clv, dtype: float64

In [40]:
cltv = cltv.reset_index()

In [41]:
cltv_final = cltv_df.merge(cltv, on='Customer ID', how='left')
cltv_final.sort_values(by='clv',ascending=False).head()

,Customer ID,recency,T,frequency,monetary,expected_purc_1_month,expected_average_profit,clv
1122,14646.0000,50.4286,50.7143,73,3646.0757,4.8119,3654.8148,55741.0845
2761,18102.0000,52.2857,52.5714,60,3859.7391,3.8636,3870.9969,47412.5801
843,14096.0000,13.8571,14.5714,17,3163.5882,2.8955,3196.4361,29061.6614
36,12415.0000,44.7143,48.2857,21,5724.3026,1.5139,5772.1782,27685.1000
1257,14911.0000,53.1429,53.4286,201,691.7101,12.4722,692.3264,27377.4115


In [42]:
# Creating Customer Segments by CLTV
cltv_final['segment'] = pd.qcut(cltv_final['clv'],4,labels=['D','C','B','A'])

In [43]:
cltv_final.head()

,Customer ID,recency,T,frequency,monetary,expected_purc_1_month,expected_average_profit,clv,segment
0,12347.0000,52.1429,52.5714,7,615.7143,0.5635,631.9123,1128.4477,A
1,12348.0000,40.2857,51.2857,4,442.6950,0.3668,463.7460,538.8089,B
2,12352.0000,37.1429,42.4286,8,219.5425,0.7271,224.8868,517.5000,B
3,12356.0000,43.1429,46.5714,3,937.1433,0.3435,995.9989,1083.0903,A
4,12358.0000,21.2857,21.5714,2,575.2100,0.4862,631.9022,966.6727,A
